<p align="center"><img src="logo/spark.png" alt="Hadoop Logo" width="250"/></p>
# **Lab 2 - Part 3 - Spark**
#### This lab will demonstrate how to perform web server log analysis with Spark. Log data is a very large, common data source and contains a rich set of information.  It comes from many sources, such as web, file, and compute servers, application logs, user-generated content,  and can be used for monitoring servers, improving business and customer intelligence, building recommendation systems, fraud detection, and much more. This lab will show you how to use Spark on real-world text-based production logs and fully harness the power of that data.

### **Part 1: Apache Web Server Log file format**


The log files that we use for this assignment are in the [Apache Common Log Format (CLF)](http://httpd.apache.org/docs/1.3/logs.html#common). The log file entries produced in CLF will look something like this:
`127.0.0.1 - - [01/Aug/1995:00:00:01 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1839`
 
Each part of this log entry is described below:
* **`127.0.0.1`:** this is the IP address (or host name, if available) of the client (remote host) which made the request to the server. 
* **`-`:** the "hyphen" in the output indicates that the requested piece of information (user identity from remote machine) is not available.
* **`-`:** the "hyphen" in the output indicates that the requested piece of information (user identity from local logon) is not available. 
* **`[01/Aug/1995:00:00:01 -0400]`:** the time that the server finished processing the request. The format is:
`[day/month/year:hour:minute:second timezone]`.
* **`"GET /images/launch-logo.gif HTTP/1.0"`:** this is the first line of the request string from the client. It consists of a three components: the request method (e.g., `GET`, `POST`, etc.), the endpoint, and the client protocol version. 
* **`200`:** this is the status code that the server sends back to the client. This information is very valuable, because it reveals whether the request resulted in a successful response (codes beginning in 2), a redirection (codes beginning in 3), an error caused by the client (codes beginning in 4), or an error in the server (codes beginning in 5). The full list of possible status codes can be found in the HTTP specification ([RFC 2616](https://www.ietf.org/rfc/rfc2616.txt) section 10).
* **`1839`:** the last entry indicates the size of the object returned to the client, not including the response headers. If no content was returned to the client, this value will be "-" (or sometimes 0).
 
Using the CLF as defined above, we create a regular expression pattern to extract the nine fields of the log line. The function returns a pair consisting of a Row object and 1. If the log line fails to match the regular expression, the function returns a pair consisting of the log line string and 0. A '-' value in the content size field is cleaned up by substituting it with 0. The function converts the log line's date string into a Python `datetime` object using the given `parse_apache_time` function.

We, then, create the primary RDD and  we'll use in the rest of this assignment. We first load the text file and convert each line of the file into an element in an RDD. Next, we use `map(parseApacheLogLine)` to apply the parse function to each element and turn each line into a pair `Row` object. Finally, we cache the RDD in memory since we'll use it throughout this notebook. The log file is available at `data/apache/apache.log`.

In [ ]:
// Just run this code
import scala.util.matching
import org.apache.spark.rdd.RDD

case class Cal(year: Int, month: Int, day: Int, hour: Int, minute: Int, second: Int)                

case class Row(host: String,
               client_identd: String,
               user_id: String,
               date_time: Cal,
               method: String,
               endpoint: String,
               protocol: String,
               response_code: Int,
               content_size: Long)
                

val month_map = Map("Jan" -> 1, "Feb" -> 2, 
                    "Mar" -> 3, "Apr" -> 4, 
                    "May" -> 5, "Jun" -> 6, 
                    "Jul" -> 7, "Aug" -> 8,  
                    "Sep" -> 9, "Oct" -> 10, 
                    "Nov" -> 11, "Dec" -> 12)

def parse_apache_time(s: String): Cal = {
    // Convert Apache time format into a Python datetime object
    // Args:
    //    s (str): date and time in Apache time format
    // Returns:
    //   datetime: datetime object (ignore timezone for now)

    return Cal(s.substring(7, 11).toInt,
            month_map(s.substring(3, 6)),
            s.substring(0, 2).toInt,
            s.substring(12, 14).toInt,
            s.substring(15, 17).toInt,
            s.substring(18, 20).toInt)
}

def parseApacheLogLine(logline: String): (Either[Row, String], Int) = {
    // Parse a line in the Apache Common Log format
    // Args:
    //    logline (str): a line of text in the Apache Common Log format
    // Returns:
    //    tuple: either a dictionary containing the parts of the Apache Access Log and 1,
    //           or the original invalid log line and 0
    
    val ret = APACHE_ACCESS_LOG_PATTERN.findAllIn(logline).matchData.toList
    if (ret.isEmpty)
        return (Right(logline), 0)

    val r = ret(0)
    val size_field = r.group(9)

    var size: Long = 0
    if (size_field != "-")
        size = size_field.toLong

    return (Left(Row(
            r.group(1),
            r.group(2),
            r.group(3),
            parse_apache_time(r.group(4)),
            r.group(5),
            r.group(6),
            r.group(7),
            r.group(8).toInt,
            size)), 1)
}

def parseLogs(): (RDD[(Either[Row, String], Int)], RDD[Row], RDD[String]) = {
    val fileName = "data/apache/apache.log"
    
    val parsed_logs = sc.textFile(fileName).map(parseApacheLogLine).cache()
    val access_logs = parsed_logs.filter(x => x._2 == 1).map(x => x._1.left.get)
    val failed_logs = parsed_logs.filter(x => x._2 == 0).map(x => x._1.right.get)

    val failed_logs_count = failed_logs.count()
    
    if (failed_logs_count > 0) {
        println(s"Number of invalid logline: $failed_logs.count()")
        failed_logs.take(20).foreach(println)
    }
    
    println(s"Read $parsed_logs.count() lines, successfully parsed $access_logs.count() lines, and failed to parse $failed_logs.count()")
    
    return (parsed_logs, access_logs, failed_logs)
}

// A regular expression pattern to extract fields from the log line
val APACHE_ACCESS_LOG_PATTERN = """^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)\s*" (\d{3}) (\S+)""".r

val (parsed_logs, access_logs, failed_logs) = parseLogs()
access_logs.take(20).foreach(println)

### **Part 2: Sample Analyses on the Web Server Log File**
 


Let's compute some statistics about the sizes of content being returned by the web server. In particular, we'd like to know what are the average, minimum, and maximum content sizes. We can compute the statistics by applying a `map` to the `access_logs` RDD. The *lambda* function we want for the map is to extract the `content_size` field from the RDD. The map produces a new RDD containing only the `content_sizes` (one element for each `Row` object in the `access_logs` RDD). To compute the minimum and maximum statistics, we can use `min()` and `max()` functions on the new RDD. We can compute the average statistic by using the `reduce` function with a *lambda* function that sums the two inputs, which represent two elements from the new RDD that are being reduced together. The result of the `reduce()` is the total content size from the log and it is to be divided by the number of requests as determined using the `count()` function on the new RDD.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
// Calculate statistics based on the content size.
val content_sizes = access_logs.map(log => log.content_size).cache()

println("Content Size Avg: " + (content_sizes.<FILL IN> / content_sizes.count()) +
         ", Min: " + content_sizes.min() +
         ", Max: " + content_sizes.max())

Next, lets look at the response codes that appear in the log. As with the content size analysis, first we create a new RDD by using a *lambda* function to extract the `response_code` field from the `access_logs` RDD. The difference here is that we will use a "pair tuple" instead of just the field itself. Using a pair tuple consisting of the response code and 1 will let us count how many records have a particular response code. Using the new RDD, we perform a `reduceByKey` function. `reduceByKey` performs a reduce on a per-key basis by applying the *lambda* function to each element, pairwise with the same key. We use the simple *lambda* function of adding the two values. Then, we cache the resulting RDD and create a list by using the `take` function.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
val responseCodeToCount = access_logs.map(log => (log.response_code, 1)).reduceByKey(<FILL IN>).cache()

val responseCodeToCountList = responseCodeToCount.take(100)

println("Found " + responseCodeToCountList.length + " response codes")
print("Response Code Counts: ")
responseCodeToCountList.foreach(x => print(x + " "))

Let's look at hosts that have accessed the server multiple times (e.g., more than ten times). First, we create a new RDD by using a *lambda* function to extract the `host` field from the `access_logs` RDD using a pair tuple consisting of the host and 1, which will let us count how many records were created by a particular host's request. Using the new RDD, we perform a `reduceByKey` function with a *lambda* function that adds the two values. We then filter the result based on the count of accesses by each host (the second element of each pair) being greater than 10. Next, we extract the host name by performing a `map` with a *lambda* function that returns the first element of each pair. Finally, we extract 20 elements from the resulting RDD.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
// Any hosts that has accessed the server more than 10 times.
val hostCountPairTuple = access_logs.map(<FILL IN>)
val hostSum = hostCountPairTuple.reduceByKey(<FILL IN>)
val hostMoreThan10 = hostSum.filter(<FILL IN>)
val hostsPick20 = hostMoreThan10.map(x => x._1).take(20)

print("Any 20 hosts that have accessed more then 10 times: ")
hostsPick20.foreach(x => print(x + " "))

For the final example, we'll look at the top endpoints (URIs) in the log. To determine them, we first create a new RDD by using a *lambda* function to extract the `endpoint` field from the `access_logs` RDD using a pair tuple consisting of the endpoint and 1 which will let us count how many records were created by a particular host's request. Using the new RDD, we perform a `reduceByKey` function with a *lambda* function that adds the two values. We then extract the top ten endpoints by performing a `takeOrdered` with a value of 10 and a *lambda* function to create a sorted list with the top endpoints at the bottom.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
// Top Endpoints
val endpointCounts = access_logs.map(<FILL IN>).reduceByKey(<FILL IN>)
val topEndpoints = endpointCounts.takeOrdered(10)(<FILL IN>)

print("Top Ten Endpoints: ")
topEndpoints.foreach(x => print(x + " "))

### **Part 3: Analyzing Web Server Log File**


What are the top ten endpoints which did not have return code 200? Create a sorted list containing top ten endpoints and the number of times that they were accessed with non-200 return code. Think about the steps that you need to perform to determine which endpoints did not have a 200 return code, how you will uniquely count those endpoints, and sort the list. 

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val not200 = access_logs.filter(<FILL IN>)
val endpointCountPairTuple = not200.map(log => (log.endpoint, 1))
val endpointSum = endpointCountPairTuple.reduceByKey(<FILL IN>)
val topTenErrURLs = endpointSum.takeOrdered(10)(<FILL IN>)

print("Top Ten failed URLs: ")
topTenErrURLs.foreach(x => print(x + " "))

Let's count the number of unique hosts in the entire log. Think about the steps that you need to perform to count the number of different hosts in the log.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val hosts = access_logs.map(<FILL IN>)
val uniqueHosts = hosts.<FILL IN>
val uniqueHostCount = uniqueHosts.count()

println("Unique hosts: " + uniqueHostCount)

Now, let's determine the number of unique hosts in the entire log on a day-by-day basis. This computation will give us counts of the number of unique daily hosts. We'd like a list sorted by increasing day of the month which includes the day of the month and the associated number of unique hosts for that day. Make sure you cache the resulting RDD `dailyHosts` so that we can reuse it in the next exercise. Think about the steps that you need to perform to count the number of different hosts that make requests *each* day. Since the log only covers a single month, you can ignore the month.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val dayToHostPairTuple = access_logs.map(<FILL IN>)
val dayGroupedHosts = dayToHostPairTuple.<FILL IN>
val dayHostCount = dayGroupedHosts.map(log => (<FILL IN>)
val dailyHosts = dayHostCount.<FILL IN>
val dailyHostsList = dailyHosts.take(30)

print("Unique hosts per day: ")
dailyHostsList.foreach(x => print(x + " "))

Next, let's determine the average number of requests on a day-by-day basis. We'd like a list by increasing day of the month and the associated average number of requests per host for that day. Make sure you cache the resulting RDD `avgDailyReqPerHost` so that we can reuse it in the next exercise. To compute the average number of requests per host, get the total number of request across all hosts and divide that by the number of unique hosts. Since the log only covers a single month, you can skip checking for the month. Also to keep it simple, when calculating the approximate average use the integer value.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val dayAndHostTuple = access_logs.map(<FILL IN>)
val groupedByDay = dayAndHostTuple.<FILL IN>
val sortedByDay = groupedByDay.<FILL IN>
val avgDailyReqPerHost = sortedByDay.map(<FILL IN>)
val avgDailyReqPerHostList = avgDailyReqPerHost.take(30)

print("Average number of daily requests per Hosts is ")
avgDailyReqPerHostList.foreach(x => print(x + " "))

### **Part 4: Exploring 404 Response Codes**


Let's count the 404 response codes. Create a RDD containing only log records with a 404 response code. Make sure you `cache()` the RDD `badRecords` as we will use it in the rest of this exercise. How many 404 records are in the log?

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val badRecords = access_logs.filter(<FILL IN>).cache()

println("Found " + badRecords.count() + " 404 URLs.")


Now, let's list the 404 response code records. Using the RDD containing only log records with a 404 response code that you cached in the previous part, print out a list up to 40 *distinct* endpoints that generate 404 errors -  no endpoint should appear more than once in your list.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val badEndpoints = badRecords.map(<FILL IN>)
val badUniqueEndpoints = badEndpoints.<FILL IN>
val badUniqueEndpointsPick40 = badUniqueEndpoints.take(40)

print("404 URLS: ")
badUniqueEndpointsPick40.foreach(x => print(x + " "))

Using the RDD containing only log records with a 404 response code that you cached before, print out a list of the top twenty endpoints that generate the most 404 errors. Remember, top endpoints should be in sorted order.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

object ErrOrdering extends Ordering[(String, Int)] {
  def compare(a: (String, Int), b: (String, Int)) = a._2 compare b._2
}

val badEndpointsCountPairTuple = badRecords.map(<FILL IN>)
val badEndpointsSum = badEndpointsCountPairTuple.reduceByKey(<FILL IN>)
val badEndpointsTop20 = badEndpointsSum.takeOrdered(20)(ErrOrdering.reverse)

print("Top Twenty 404 URLs: ")
badEndpointsTop20.foreach(x => print(x + " "))

Instead of looking at the endpoints that generated 404 errors, let's look at the hosts that encountered 404 errors. Using the RDD containing only log records with a 404 response code that you cached before, print out a list of the top twenty-five hosts that generate the most 404 errors.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val errHostsCountPairTuple = badRecords.map(<FILL IN>)
val errHostsSum = errHostsCountPairTuple.reduceByKey(<FILL IN>)
val errHostsTop25 = errHostsSum.takeOrdered(25)(Ordering[Int].reverse.on(x => x._2))

print("Top 25 hosts that generated errors: ")
errHostsTop25.foreach(x => print(x + " "))

Let's explore the 404 records temporally. Break down the 404 requests by day and get the daily counts sorted by day as a list. Since the log only covers a single month, you can ignore the month in your checks.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val errDateCountPairTuple = badRecords.map(<FILL IN>)
val errDateSum = errDateCountPairTuple.reduceByKey(<FILL IN>)
val errDateSorted = errDateSum.cache()
val errByDate = errDateSorted.takeOrdered(30)

print("404 Errors by day: ")
errByDate.foreach(x => print(x + " "))

errDateSorted.cache()

Using the RDD `errDateSorted` you cached before, what are the top five days for 404 response codes and the corresponding counts of 404 response codes?

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

object DateOrdering extends Ordering[(Int, Int)] {
  def compare(a: (Int, Int), b: (Int, Int)) = a._2 compare b._2
}

val topErrDate = errDateSorted.<FILL IN>

print("Top Five dates for 404 requests: ")
topErrDate.foreach(x => print(x + " "))

Using the RDD `badRecords` you cached before, and by hour of the day and in increasing order, create an RDD containing how many requests had a 404 return code for each hour of the day (midnight starts at 0). Cache the resulting RDD hourRecordsSorted and print that as a list.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val hourCountPairTuple = badRecords.map(<FILL IN>)
val hourRecordsSum = hourCountPairTuple.reduceByKey(<FILL IN>)
val hourRecordsSorted = hourRecordsSum.<FILL IN>.cache()
val errHourList = hourRecordsSorted.collect()

print("Top hours for 404 requests: ")
errHourList.foreach(x => print(x + " "))